# Imports and packages

In [1]:
import pandas as pd
import numpy as np
import random
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
from torchinfo import summary

c:\Users\sandr\anaconda3\envs\tccenv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [87]:
import os

CUDA_VISIBLE_DEVICES=""
os.environ["CUDA_VISIBLE_DEVICES"]=""


# Dataprep

In [3]:
# tirar @ e links

def removeMention(tweet):
  words = tweet.split()
  words = [word for word in words if "@" not in word ]
  newTweet = ' '.join(words)
  return newTweet

  # substituir menção por <USER>


In [4]:
import emoji
from emoji import UNICODE_EMOJI

In [5]:
# search your emoji
def is_emoji(s, language="en"):
    return s in UNICODE_EMOJI[language]

# add space near your emoji
def add_space(text):
    return ''.join(' ' + char + ' ' if is_emoji(char) else char for char in text).strip()

def separateEmoji(tweet):
  return add_space(tweet)

In [6]:
import re
regex = re.compile("(http://t\.co.{12})|(https://t\.co.{11})")

def removeLink(tweet):
  #words = tweet.split()
  #links = [word for word in words if "t.co" in word]
  words = regex.sub('',tweet)
  return words

# LSTM

# Bag of words

In [7]:
from collections import Counter


def tweets_vocab(dataset, n=None):
    if n is None:
        n = len(dataset)
    all_tweets = []
    for tweet in dataset:
        words = tweet.split()
        all_tweets.append(words)

    all_tweets = [item for sublist in all_tweets for item in sublist]
    #print(all_tweets)
    counter_words = Counter(all_tweets)
    #print(counter_words)
    #print(len(counter_words))
    counter_words = counter_words.most_common(n)
    vocab_to_int = {w:i+1 for i, (w,c) in enumerate(counter_words)}
    #print(vocab_to_int)
    return vocab_to_int
  
def padding(dataset_tok, max_len):
  for i in range(len(dataset_tok)):
      if len(dataset_tok[i]) < max_len:
      #if len(dataset_tok[i]) < max_len:
          for z in range(max_len - len(dataset_tok[i])):
          #for z in range(max_len - len(tweet)):
              # dataset_tok[i].append([0])
              dataset_tok[i] = [[0]] + dataset_tok[i]
              #tweet.append(0)
  return np.array(dataset_tok)

def tweets_tok(dataset, vocab_to_int):

    dataset_tok = []
    max_len = 0

    # for tweet in dataset_tok:
    #   print(tweet)
    #     r = [vocab_to_int[w] if w in vocab_to_int else 0 for w in tweet.split()]
    #     dataset_tok[tweet]=r
    #     #print(r)
    #     #tweet = r
    for i in range(len(dataset)):
        r = [[vocab_to_int[w]] for w in dataset.iloc[i].split() if w in vocab_to_int]
        dataset_tok.append(r)
        #dataset_tok.iloc[i] = r

        if len(r) > max_len:
            max_len = len(r)
        #print(r)
        #tweet = r
    
    # # padding
    # for i in range(len(dataset_tok)):
    #     #if len(dataset_tok.iloc[i]) < max_len:
    #     if len(dataset_tok[i]) < max_len:
    #         #for z in range(max_len - len(dataset_tok.iloc[i])):
    #         for z in range(max_len - len(dataset_tok[i])):
    #             #dataset_tok.iloc[i].append(0)
    #             dataset_tok[i].append(0)

    return dataset_tok, max_len

In [8]:
train_x_clean = train_x.apply(lambda x:separateEmoji(x)).apply(lambda x:removeMention(x)).apply(lambda x:removeLink(x))
test_x_clean = covid_test['text'].apply(lambda x:separateEmoji(x)).apply(lambda x:removeMention(x)).apply(lambda x:removeLink(x))
train_x_clean

NameError: name 'train_x' is not defined

In [9]:
vocab_to_int = tweets_vocab(train_x_clean)
train_x_tok, max_len1 = tweets_tok(train_x_clean, vocab_to_int)
#print(train_x_tok)

NameError: name 'train_x_clean' is not defined

In [10]:
test_x_tok, max_len2 = tweets_tok(test_x_clean, vocab_to_int)
#print(test_x_tok)

NameError: name 'test_x_clean' is not defined

In [ ]:
print(test_x_tok[0])
max_len = max_len2
if max_len1 > max_len2:
  max_len = max_len1



In [ ]:
train_x_tok = padding(train_x_tok, max_len)
test_x_tok = padding(test_x_tok, max_len)
#train_x_tok.head()
print(train_x_tok)
#print(test_x_tok)

[[[   0]
  [   0]
  [   0]
  ...
  [  14]
  [ 606]
  [ 308]]

 [[   0]
  [   0]
  [   0]
  ...
  [  16]
  [ 468]
  [  20]]

 [[   0]
  [   0]
  [   0]
  ...
  [1248]
  [1249]
  [ 687]]

 ...

 [[   0]
  [   0]
  [   0]
  ...
  [ 181]
  [ 761]
  [1501]]

 [[   0]
  [   0]
  [   0]
  ...
  [ 132]
  [ 109]
  [1730]]

 [[   0]
  [   0]
  [   0]
  ...
  [ 493]
  [ 115]
  [  44]]]


In [ ]:
torch.from_numpy(train_y.to_numpy()).to(torch.int64)

tensor([1, 1, 1,  ..., 0, 0, 1])

In [ ]:
print(train_y)
print(train_y.unique())

0       1.0
1       1.0
2       1.0
3       2.0
4       1.0
       ... 
2283    0.0
2286    0.0
2287    0.0
2288    0.0
2289    1.0
Name: label, Length: 1995, dtype: float64
[1. 2. 0.]


In [ ]:
print(train_y.value_counts())
print(covid_test['BERT_label'].value_counts())

2.0    752
1.0    685
0.0    558
Name: label, dtype: int64
1    161237
2    129636
0     85516
Name: BERT_label, dtype: int64


In [ ]:
covid_test[covid_test['BERT_label']==1].count()

index         161237
Tweet ID      161237
text          161237
BERT_label    161237
dtype: int64

In [ ]:
train_y = train_y.mask(train_y < 2, 0).astype(int)
train_y = train_y.mask(train_y == 2, 1).astype(int)

test_y = covid_test['BERT_label']
test_y = test_y.mask(test_y < 2, 0).astype(int)
test_y = test_y.mask(test_y == 2, 1).astype(int)

# test_y = covid_test['BERT_label'].mask(covid_test['BERT_label'] < 2, 0).astype(int)
# test_y = covid_test['BERT_label'].mask(covid_test['BERT_label'] == 2, 1).astype(int)

print(train_y.unique())
print(test_y.unique())

[0 1]
[0 1]


In [ ]:
print(train_y.value_counts())
print(test_y.value_counts())

0    1243
1     752
Name: label, dtype: int64
0    246753
1    129636
Name: BERT_label, dtype: int64


In [ ]:
print(type(train_y[0]))
print(type(test_y[0]))

<class 'pandas.core.series.Series'>
<class 'numpy.int64'>


In [ ]:
378384

In [8]:
# train_x_comp = np.load('train_x.npy')
train_y = np.load('train_y.npy')
# val_x_tok = np.load('val_x.npy')
val_y = np.load('val_y.npy')
# test_x_tok = np.load('test_x.npy')
test_y = np.load('test_y.npy')

In [9]:
train_x_tok = pd.Series(np.load('train_words_x.npy', allow_pickle=True))
# print(test_x_tok.head())
train_x_tok = train_x_tok.apply(lambda x:separateEmoji(x)).apply(lambda x:removeMention(x)).apply(lambda x:removeLink(x))
# vocab_to_int = tweets_vocab(train_x_tok, 200)
# train_x_tok_2 = [i.split() for i in train_x_tok]
train_x_tok


0         Oh you people are unbelievably stupid why the ...
1         Kazumi: "Not all Asian women have flat asses. ...
2         BTS Wordle 220525 2/6 🟪 ⬛ ️ ⬛ ️ 🟪 ⬛ ️ 🟪 🟪 🟪 🟪 ...
3         Theres all types of privilege u jus dnt recogn...
4         Do you think Canberra, and the rest of the wor...
                                ...                        
303102    It was my Granddaughters 2nd birthday at the w...
303103    I don't think that chart shows identical figur...
303104         Delete this. It ain’t funny and it’s racist.
303105                                       Not this again
303106    Who needs 'em. DiSantis knows COVID is fake ne...
Length: 303107, dtype: object

In [10]:
t = [['a', 'b', 'c', 'd'],['a', 'b', 'c'],['a', 'b', 'd']]
u = [[word for word in l if word in(['a','b','c'])] for l in t]
print(u)

[['a', 'b', 'c'], ['a', 'b', 'c'], ['a', 'b']]


In [11]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import CountVectorizer

In [12]:
cntvct = CountVectorizer(max_features=2000)
train_x_tok_3 = cntvct.fit_transform(train_x_tok)


In [13]:
svd = TruncatedSVD(n_components=200, random_state=42)
train_x_svd = svd.fit_transform(train_x_tok_3)

In [14]:
test_x_tok = pd.Series(np.load('test_words_x.npy', allow_pickle=True))
# print(test_x_tok.head())
test_x_tok = test_x_tok.apply(lambda x:separateEmoji(x)).apply(lambda x:removeMention(x)).apply(lambda x:removeLink(x))
# test_x_tok = [i.split() for i in test_x_tok]

val_x_tok = pd.Series(np.load('val_words_x.npy', allow_pickle=True))
# print(test_x_tok.head())
val_x_tok = val_x_tok.apply(lambda x:separateEmoji(x)).apply(lambda x:removeMention(x)).apply(lambda x:removeLink(x))
# val_x_tok = [i.split() for i in val_x_tok]


In [15]:
val_x_tok_3 = cntvct.transform(val_x_tok)
test_x_tok_3 = cntvct.transform(test_x_tok)


In [16]:
val_x_svd = svd.transform(val_x_tok_3)
test_x_svd = svd.transform(test_x_tok_3)

In [17]:
test_x_svd.shape

(37639, 200)

In [20]:
np.expand_dims(train_x_svd, axis=2)

array([[[ 2.19055459e+00],
        [-1.29294448e+00],
        [ 2.60391923e+00],
        ...,
        [ 2.22318661e-02],
        [ 2.45931310e-04],
        [ 3.25665555e-02]],

       [[ 2.56397018e-01],
        [-1.48115728e-01],
        [ 2.48622158e-02],
        ...,
        [ 5.21197792e-02],
        [-1.26416102e-01],
        [ 4.05924299e-02]],

       [[ 5.15309733e-01],
        [-2.09957723e-02],
        [-1.28170289e-01],
        ...,
        [-2.43712375e-02],
        [-2.39855146e-02],
        [ 8.02499397e-02]],

       ...,

       [[ 7.50929123e-01],
        [-5.69521894e-01],
        [ 1.42043563e-01],
        ...,
        [-3.77241034e-02],
        [ 4.80523140e-03],
        [-5.58832336e-03]],

       [[ 1.93942152e-01],
        [-1.38873489e-01],
        [ 8.30667279e-03],
        ...,
        [ 9.30617252e-02],
        [-2.98220194e-02],
        [ 2.97058794e-02]],

       [[ 1.66959434e+00],
        [-4.82385266e-01],
        [ 1.58670031e+00],
        ...,
        

In [21]:
#train_1_ds = TensorDataset(torch.from_numpy(train_x_tok).type(torch.float32), torch.from_numpy(train_y.to_numpy()))
#train_2_ds = TensorDataset(torch.from_numpy(train_x_tok_2).type(torch.float32), torch.from_numpy(train_y_2))
train_ds = TensorDataset(torch.from_numpy(np.expand_dims(train_x_svd, axis=2)).type(torch.float32), torch.from_numpy(train_y))
#train_ds = torch.utils.data.ConcatDataset([train_1_ds, train_2_ds])
val_ds = TensorDataset(torch.from_numpy(np.expand_dims(val_x_svd, axis=2)).type(torch.float32), torch.from_numpy(val_y))
test_ds = TensorDataset(torch.from_numpy(np.expand_dims(test_x_svd, axis=2)).type(torch.float32), torch.from_numpy(test_y))

batch_size = 256
train_dl = DataLoader(train_ds, shuffle=True, batch_size=batch_size)
val_dl = DataLoader(val_ds, shuffle=True, batch_size=batch_size)
test_dl = DataLoader(test_ds, shuffle=True, batch_size=batch_size)

In [5]:
max_len=128
vocab_size=1996

# LSTM model

In [88]:
class TweetsLSTM(nn.Module):
    def __init__(self,no_layers,hidden_dim,input_dim,drop_prob=0.5):
        super(TweetsLSTM,self).__init__()
 
        #self.output_dim = output_dim
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.no_layers = no_layers
    
        print(type(self.no_layers), type(self.input_dim), type(self.hidden_dim))
        
        # embedding and LSTM layers
        #self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        #lstm
        self.lstm_layers = nn.LSTM(input_size=self.input_dim,hidden_size=self.hidden_dim,
                           num_layers=no_layers, batch_first=True)
        
        
        # dropout layer
        self.dropout_layer = nn.Dropout(p=drop_prob, inplace=True)
    
        # linear and sigmoid layer
        self.linear_layer = nn.Linear(self.hidden_dim, 1)
        self.sigmoid_layer = nn.Sigmoid()

    def forward(self,x,hidden):
        print("x:", x.shape, x.dtype)
        x = x.to(torch.device('cpu'))
        h = hidden[0].to(torch.device('cpu'))
        print(h.device)
        c = hidden[1].to(torch.device('cpu'))
        batch_size, _seq1 , _seq2 = x.size()
        print(batch_size)
        
        # h_1 = torch.zeros(self.no_layers, batch_size, self.hidden_dim, device=torch.device('cpu'))
        # c_1 = torch.zeros(self.no_layers, batch_size, self.hidden_dim, device=torch.device('cpu'))
        
        # hc_1 = (h_1, c_1)
        
        print("hidden:", hidden[0].device, x.device)
        # embeddings and lstm_out
        #embeds = self.embedding(x)  # shape: B x S x Feature   since batch = True
        #print("emb:",embeds.shape)  #[50, 500, 1000]
        #print(embeds[0])
        #print("emb: ",embeds)

        lstm_out, (h,c) = self.lstm_layers(x, (h, c))
        print("lstm out:",lstm_out.shape, hidden.device)
        #lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim) 
        
        # dropout and fully connected layer
        out = self.dropout_layer(h) #(h_1) lstm_out
        #print(out.contiguous().view(batch_size,-1))
        #out = self.fc(out.contiguous().view(batch_size,-1))
        #print("out:", out,out.shape)
        lin_out = self.linear_layer(out)
        #print("lin_out: ", lin_out.shape)
        
        
        # sigmoid function
        sig_out = self.sigmoid_layer(lin_out)
        #print("out2:", sig_out.shape)
        # reshape to be batch_size first
        #sig_out = sig_out.view(batch_size, -1)
        #print("sig_out: ", sig_out.shape)

        #sig_out = sig_out[:, -1]
        #print(sig_out)
        # # reshape to be batch_size first
        #sig_out = out.view(batch_size, -1)
        #print("out3:", sig_out.shape)
        #sig_out = sig_out[:, -1] # get last batch of labels
        #print("sig_out:", sig_out.shape)
        
        #sig_out = out.view(batch_size,-1)
        sig_out = sig_out[-1,:,:]
        #print(sig_out.shape,sig_out)
        # # return last sigmoid output and hidden state

        #sig_out = torch.max(out,1)
        # print("sig_out:",sig_out.shape, sig_out)
        #sig_out = torch.max(sig_out,1)
        #print("sig_out:",sig_out)
        #return sig_out[1], hidden
        return sig_out, hidden

    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        h0 = torch.zeros(self.no_layers,batch_size,self.hidden_dim,device=torch.device('cpu'))
        c0 = torch.zeros(self.no_layers,batch_size,self.hidden_dim,device=torch.device('cpu'))
        hidden = (h0,c0)
        return hidden

In [89]:
no_layers = 2
#vocab_size = len(vocab_to_int) + 1 #extra 1 for padding
#vocab_size = 128
embedding_dim = 64
input_dim = 1#200
output_dim = 3
hidden_dim = 50 #256


model = TweetsLSTM(no_layers,hidden_dim,input_dim,drop_prob=0.3).to(torch.device('cpu'))

#print(train_x_tok.shape)
dataiter = iter(train_dl)
print('sample input: ', dataiter.next()[0].size())
print('sample output: ', dataiter.next()[1].size())
print(model)
#summary(model, (4, 200, 1),dtypes=[torch.int])
dev = torch.device('cpu')
summary(model, input_data=[torch.tensor(np.ones((4, 200, 1))).type(torch.float32).to(dev), (torch.tensor(np.zeros((2, 4, 50))).to(dev), torch.tensor(np.zeros((2, 4, 50))).to(dev))])

<class 'int'> <class 'int'> <class 'int'>
sample input:  torch.Size([256, 200, 1])
sample output:  torch.Size([256])
TweetsLSTM(
  (lstm_layers): LSTM(1, 50, num_layers=2, batch_first=True)
  (dropout_layer): Dropout(p=0.3, inplace=True)
  (linear_layer): Linear(in_features=50, out_features=1, bias=True)
  (sigmoid_layer): Sigmoid()
)
x: torch.Size([4, 200, 1]) torch.float32
cpu
4
hidden: cuda:0 cpu


RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: []

In [46]:
# loss and optimization functions
lr=0.001

def BCELoss_class_weighted(weights):
  def loss(input, target):
    input = torch.clamp(input,min=1e-7,max=1-1e-7)
    bce = - weights[1] * target * torch.log(input) - (1 - target) * weights[0] * torch.log(1 - input)
    return torch.mean(bce)
  return loss

weights = [len(train_y)/(2*(len(train_y)-train_y.sum())), len(train_y)/(2*train_y.sum())]
criterion = BCELoss_class_weighted(weights)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# function to predict accuracy
def acc(pred,label):
    #pred = torch.round(pred.squeeze())
    #print(pred, label, torch.sum(pred == label.squeeze()).item())
    return torch.sum(((pred >= 0.5).type(torch.float32) == label.squeeze()).type(torch.float32)).item()

In [14]:
a = torch.tensor(np.array([1, 0.6, 0.3, 0]))
b = torch.tensor(np.array([1, 0, 0, 0]))
torch.sum(((a >= 0.5).type(torch.int) == b).type(torch.float32)).item()

3.0

In [15]:
def training_loop(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  running_loss = 0

  model.train()
  for batch, (X, y) in enumerate(dataloader):
    optimizer.zero_grad()
    pred = model(X)
    #final_pred = torch.squeeze((pred >= 0.5).type(torch.int),1)
    final_pred = torch.squeeze(pred, 1).type(torch.float32)
    #y = torch.unsqueeze(y, 0)
    #print(final_pred, y)
    loss = loss_fn(final_pred, y.type(torch.float32))

    loss.backward()
    optimizer.step()

    running_loss = running_loss + loss.item()

    if batch % 100 == 0:
      loss, current = loss.item(), batch*len(X)
      print(f'loss: {loss:>7f}  [{current:>5d}/{size:>5d}]')
    
  return running_loss

In [ ]:
def testing_loop(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  test_loss, correct = 0, 0
  running_loss = 0

  model.eval()
  with torch.no_grad():
    for X, y in dataloader:
      pred = model(X)
      #final_pred = (pred >= 0.5).type(torch.int)
      final_pred = torch.squeeze(pred, 1).type(torch.float32)
      test_loss += loss_fn(final_pred, y.type(torch.float32)).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
      running_loss = running_loss + test_loss
  
  test_loss /= num_batches
  correct /= size
  print(f'Test Error: \n Accuracy:{(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n')

  return running_loss

In [ ]:
epochs = 50
best_val_loss = 100000000
for i in range (epochs):
  tr_loss = training_loop(train_dl, model, criterion, optimizer)
  val_loss = testing_loop(val_dl, model, criterion)
  if val_loss < best_val_loss:
    best_val_loss = val_loss
    torch.save(model.state_dict(), '/content/drive/MyDrive/poli 2022/new_model')

In [61]:
clip = 5
epochs = 50
valid_loss_min = np.Inf
# train for some number of epochs
epoch_tr_loss,epoch_vl_loss = [],[]
epoch_tr_acc,epoch_vl_acc = [],[]
print(len(train_dl.dataset))
print(len(val_dl.dataset))

for epoch in range(epochs):
    print("epoch:", epoch)
    train_losses = []
    train_acc = 0.0
    model.train()
    # initialize hidden state 
    h = model.init_hidden(batch_size)
    for inputs, labels in train_dl:
        print(inputs.device, h[0].device)
        
        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])
        optimizer.zero_grad()
        print('ok')
        output, h = model(inputs, h)
        
        # calculate the loss and perform backprop
        #print("output:",output.shape, output[0].shape, output[0].dtype)
        #print("labels:",labels.shape, labels.dtype)
        #print(output, torch.max(labels,1)[1])
        #print(output.T, labels.type(torch.float32))
        loss = criterion(torch.squeeze(output, 1).type(torch.float32), labels.type(torch.float32))
        loss.backward()
        train_losses.append(loss.item())
        # calculating accuracy
        #accuracy = acc(output,labels)
        accuracy = acc(torch.squeeze(output, 1).type(torch.float32), labels.type(torch.float32))
        train_acc += accuracy
        #`clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        #nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
    
    val_h = model.init_hidden(batch_size)
    val_losses = []
    val_acc = 0.0
    model.eval()
    for inputs, labels in val_dl:
            print(inputs.device, val_h.device)
            val_h = tuple([each.data for each in val_h])
            output, val_h = model(inputs, val_h)
            #print("output:",output.squeeze())
            #print("labels:",labels.long())
            val_loss = criterion(torch.squeeze(output, 1).type(torch.float32), labels.type(torch.float32))

            val_losses.append(val_loss.item())
            
            accuracy = acc(torch.squeeze(output, 1).type(torch.float32), labels.type(torch.float32))
            val_acc += accuracy
            epoch_train_loss = np.mean(train_losses)
    epoch_val_loss = np.mean(val_losses)
    epoch_train_acc = train_acc/len(train_dl.dataset)
    epoch_val_acc = val_acc/len(val_dl.dataset)
    epoch_tr_loss.append(epoch_train_loss)
    epoch_vl_loss.append(epoch_val_loss)
    epoch_tr_acc.append(epoch_train_acc)
    epoch_vl_acc.append(epoch_val_acc)
    print(f'Epoch {epoch+1}') 
    print(f'train_loss : {epoch_train_loss} val_loss : {epoch_val_loss}')
    print(f'train_accuracy : {epoch_train_acc*100} val_accuracy : {epoch_val_acc*100}')
    if epoch_val_loss <= valid_loss_min:
        torch.save(model.state_dict(), 'model_emb')
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,epoch_val_loss))
        valid_loss_min = epoch_val_loss
    print(25*'==')

303107
37638
epoch: 0
cpu cpu
ok
x: torch.Size([256, 200, 1]) torch.float32
256


RuntimeError: Input and parameter tensors are not at the same device, found input tensor at cpu and parameter tensor at cuda:0

In [ ]:
model = TweetsLSTM(no_layers,hidden_dim,input_dim,drop_prob=0.5)
model.load('/content/drive/MyDrive/poli 2022/model')
model.eval()

In [ ]:
np.empty((1,0))

array([], shape=(1, 0), dtype=float64)

In [18]:
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

def predict_text(inputs):
      output = model(inputs)
      return output

corrects = 0
total = 0
TP = 0
TN = 0
FP = 0
FN = 0
all_pred_bin = np.empty((1,0))
all_labels = np.empty((1,0))

for inputs, labels in test_dl:
  #print(inputs.shape)
  pred_test = predict_text(inputs)
  #print(torch.squeeze(pred_test, 1).type(torch.float32))
  all_pred_bin = np.append(all_pred_bin, (torch.squeeze(pred_test, 1).type(torch.float32) >= 0.5).type(torch.int).detach().numpy())
  all_labels = np.append(all_labels, labels.numpy())
  #print(torch.squeeze(pred_test.T, 0), labels.type(torch.float32))
  corrects += acc(torch.squeeze(pred_test, 1).type(torch.float32), labels.type(torch.float32))#acc(pred_test, labels)
  #print(corrects)
  total += len(pred_test)
print(corrects/total)
print(all_pred_bin.shape, all_pred_bin)
print(all_labels.shape, all_labels)
print("f1: ", f1_score(all_labels, all_pred_bin))
print("recall: ", recall_score(all_labels, all_pred_bin))
print("precision: ", precision_score(all_labels, all_pred_bin))
print("accuracy: ", accuracy_score(all_labels, all_pred_bin))

0.7760301814607189
(37639,) [1. 0. 1. ... 1. 1. 0.]
(37639,) [1. 0. 0. ... 0. 1. 0.]
f1:  0.704625087596356
recall:  0.778250773993808
precision:  0.6437259923175416
accuracy:  0.7760301814607189


In [ ]:
print(all_pred_bin.sum())

37639.0
